In [121]:
import numpy as np
import time
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure 
from bokeh.layouts import row, column
# from bokeh.models import Label
# from ipywidgets import interact
output_notebook()

Loading BokehJS ...

In [127]:
# Initial values
dist_size = 30
car_size = 1
car1_initpos = 4
car2_initpos = 0
car1_vel = 4
car2_initvel = 2
car1_acc = 0
car2_initacc = 0
car2_maxacc = 2
car2_maxbrake = 40
dt = 0.015
timesteps = int((dist_size - car1_initpos) / car1_vel / dt)
dist_sep = 2 * car_size
resp = 0
resp_kf = 0

timepoints = np.zeros(timesteps)
poscar1 = np.zeros(timesteps)
poscar1_est = np.zeros(timesteps)
poscar1_meas = np.zeros(timesteps)
poscar2 = np.zeros(timesteps)
car_sep = np.zeros(timesteps)
car2_acc = np.zeros(timesteps)
poscar2_kf = np.zeros(timesteps)
car_sep_kf = np.zeros(timesteps)
car2_acc_kf = np.zeros(timesteps)
noiserange = np.zeros(timesteps)
pdf_proc = np.zeros(timesteps)
pdf_meas = np.zeros(timesteps)

car1_start_pos = [3, 4, 5, 2]
car2_start_vel = [2, 3, 4, 2]
car2_acc_int = np.zeros(len(car1_start_pos))
car2_acc_kf_int = np.zeros(len(car1_start_pos))
car2_resp = np.zeros(len(car1_start_pos))
car2_kf_resp = np.zeros(len(car1_start_pos))

In [128]:
def RunCarFollowing(PN=0.4, MN=0.6, car1_initpos=3, car2_initvel=4):
    
    # Setup initial values
    pos1real = car1_initpos
    pos2 = car2_initpos
    pos2_kf = car2_initpos
    vel2 = car2_initvel
    vel2_kf = car2_initvel
    acc2 = car2_initacc
    acc2_kf = car2_initacc
    tt = 0
    resp_flag = False
    resp_kf_flag = False
    
    # Initialize Kalman Filter
    f = KalmanFilter(dim_x=2, dim_z=1)
    f.x = np.array([[car1_initpos],  # position
                    [car1_vel]])  # velocity
    f.F = np.array([[1.0, 1.0],
                    [0.0, 1.0]])
    f.H = np.array([[1.0, 0.0]])
    f.P = np.array([[1000.0, 0.0],
                    [0.0, 1000.0]])
    f.R = np.array([[5.0]])
    f.Q = Q_discrete_white_noise(dim=2, dt=dt, var=PN**2)  # Process noise
    
    # Loop for updating positions
    for tt in range(timesteps):

        # Real position of leading car
        pos1real = pos1real + car1_vel * dt

        # Measurement of position of leading car with noise
        z = np.random.normal(pos1real, MN)

        # Kalman Filter predict and update steps
        f.predict()
        f.update(z)   

        # Following car update without Kalman Filter
        dx = z - pos2
        err = (dx - dist_sep) / dist_sep
        
        if tt > int(timesteps / 5) and resp_flag == False:
            if err < 0.03 and err > -0.03:
                global resp
                resp = tt * dt
                resp_flag = True
            
        if err > 1.0: err = 1.0
        # Acceleration model
        if err > 0:
            acc2 = err * car2_maxacc
        else:
            acc2 = err * car2_maxbrake
        vel2 = vel2 + acc2 * dt
        # crash protection 
        if vel2 < 0:
            vel2 = 0
            acc2 = 0
        # update position
        pos2 = pos2 + vel2 * dt + 0.5 * acc2 * dt**2

        # Following car update with Kalman Filter
        dx_kf = f.x[0, 0] - pos2_kf
        err_kf = (dx_kf - dist_sep) / dist_sep
        
        if tt > int(timesteps / 5) and resp_kf_flag == False:
            if err_kf < 0.03 and err > -0.03:
                global resp_kf
                resp_kf = tt * dt
                resp_kf_flag = True
        
        if err_kf > 1.0: err_kf = 1.0
        # Acceleration model
        if err_kf > 0:
            acc2_kf = err_kf * car2_maxacc
        else:
            acc2_kf = err_kf * car2_maxbrake
        vel2_kf = vel2_kf + acc2_kf * dt
        # crash protection
        if vel2_kf < 0:
            vel2_kf = 0
            acc2_kf = 0
        # update position
        pos2_kf = pos2_kf + vel2_kf * dt + 0.5 * acc2_kf * dt**2

        # Update arrays
        global timepoints, poscar1, poscar2, poscar1_est, car_sep
        timepoints[tt] = tt * dt
        poscar1[tt] = pos1real
        poscar1_est[tt] = f.x[0, 0]
        poscar1_meas[tt] = z
        poscar2[tt] = pos2
        poscar2_kf[tt] = pos2_kf
        car_sep[tt] = pos1real - pos2
        car_sep_kf[tt] = pos1real - pos2_kf  
        car2_acc[tt] = acc2
        car2_acc_kf[tt] = acc2_kf

In [131]:
for n in range(len(car1_start_pos)):
    RunCarFollowing(0.6, 0.6, car1_start_pos[n], car2_start_vel[n])
    global car2_acc_int, car2_acc_kf_int
    car2_acc_int[n] = np.sum(np.abs(car2_acc))
    car2_acc_kf_int[n] = np.sum(np.abs(car2_acc_kf))
    car2_resp[n] = resp
    car2_kf_resp[n] = resp_kf

p1 = figure(plot_width=500, plot_height=400, x_axis_label='Index', y_axis_label='energy (arb. units)',
            title='Energy trade off')
p1.line(range(len(car1_start_pos)), car2_acc_int, line_color='gold', line_width=2, legend='without KF')
p1.line(range(len(car1_start_pos)), car2_acc_kf_int, line_color='crimson', line_width=2, legend='with KF')
p2 = figure(plot_width=500, plot_height=400, x_axis_label='Index', y_axis_label='response (s)',
            title='Response time trade off')
p2.line(range(len(car1_start_pos)), car2_resp, line_color='gold', line_width=2, legend='without KF')
p2.line(range(len(car1_start_pos)), car2_kf_resp, line_color='crimson', line_width=2, legend='with KF')
show(row(p1, p2))